In [1]:
from tkinter import *
from tkinter import messagebox
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from smtplib import SMTP_SSL
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.header import Header
from smtplib import SMTPRecipientsRefused
import time
import threading

In [2]:
class application(Frame):
    def __init__(self, master = None):
        super().__init__(master)
        self.master = master
        self.pack()
        self.createWidget()

    def createWidget(self):
        self.config(bg="#000000")
        self.label01 = Label(self, text = "Exchange rate Auto reminder", width= 600, height= 2, bg = "black",fg = "yellow",font=("华文行楷",30,"bold"))
        self.label01.pack()
        self.label02 = Label(self, text = "Please input your email address",width= 600,height= 2,bg = "black",fg = "white", font=("Times",15,"bold italic"))
        self.label02.pack()
        v1 = StringVar()
        self.entry01 = Entry(self,textvariable=v1,bd=5)
        self.entry01.pack()
        self.label03 = Label(self, text = "Please input expected exchange rate(100AUD = ?CNY)",width= 600,height= 2,bg = "black",fg = "white", font=("Times",15,"bold italic"))
        self.label03.pack()
        v2 = DoubleVar()
        self.entry02 = Entry(self,textvariable=v2,bd=5)
        self.entry02.pack()
        self.label04 = Label(self, text = "Please enter the time interval(min)",width= 600,height= 2,bg = "black",fg = "white", font=("Times",15,"bold italic"))
        self.label04.pack()
        v3 = DoubleVar()
        self.entry03 = Entry(self,textvariable=v3,bd=5)
        self.entry03.pack()
        self.label05 = Label(self, text = "Designed by Ezra",width= 600,height= 1,bg = "black",fg = "white",font=("华文行楷",10))
        self.label05.pack()
        self.btn01 = Button(self,text = "Check",command = self.check,bg = "black",fg = "white",bd = 5,relief = "groove", activebackground = "grey",font = ("Times",20,"bold italic"))
        self.btn01.pack(side = "left",expand = "True")
        #self.btn01.place(relx=0.2,rely=0.99)
        self.btn02 = Button(self,text = "Send",command = lambda:self.thread_it(self.deliver),bg = "black",fg = "white",bd = 5,relief = "groove", activebackground = "grey",font = ("Times",20,"bold italic"))
        self.btn02.pack(side = "left",expand = "True")
        self.btn03 = Button(self,text = "Stop",command = self.stoptd,bg = "black",fg = "white", borderwidth=5,relief = "groove", activebackground = "grey",font = ("Times",20,"bold italic"))
        self.btn03.pack(side = "left",expand = "True")
        self.btnQuit = Button(self,text = "Exit", command= root.destroy, bg = "black",fg = "white",borderwidth=5,relief = "groove", activebackground = "grey",font = ("Times",20,"bold italic"))
        self.btnQuit.pack(side = "left",expand = "True")

    def thread_it(self,func, *args):
        global stop
        stop = False
        t = threading.Thread(target=func, args=args) 
        t.setDaemon = True
        t.start()
    def stoptd(self):
        global stop
        stop = True
        messagebox.showinfo("Reminder","Successfully Stopped ")

    def check(self):
        mail = self.entry01.get()
        threshold = self.entry02.get()
        mim = self.entry03.get()
        try:
            threshold = float(threshold)
            try:
                sec = float(mim)*60
                if mail.count("@") == 1:
                    messagebox.showinfo("Reminder","Completed")
                else:
                    messagebox.showwarning("Error","Please enter the correct format of email")
            except:
                messagebox.showwarning("Error","incorrect format, please enter minutes i.e. (60)")
        except:
            messagebox.showwarning("Error","incorrect format, please enter integer or float i.e (470)")
    def getrate(self):
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        #chrome_options.add_argument('--disable-gpu')
        driver = webdriver.Chrome(options=chrome_options)
        driver.get("http://www.bankcomm.com/BankCommSite/shtml/jyjr/cn/7158/7161/8091/list.shtml?channelId=7158")
        frame = driver.find_elements(By.TAG_NAME,'iframe')[0]
        driver.switch_to.frame(frame)
        table = driver.find_elements(By.TAG_NAME,'tr')
        str = table[5].text
        AUD = str.split()
        rate = float(AUD[3])
        return rate
    def ctt(self,rate):
        dt = time.strftime("%Y-%m-%d %H:%M:%S",time.localtime())
        mail_content = "Today's exchange rate is 1AUD="+ str(rate/100) + "CNY. " + dt
        return mail_content
    def deliver(self):
        i=0
        if self.entry01.get().count("@") != 1:
            messagebox.showinfo("Error","Incorrect email address")
        else:
            messagebox.showinfo("Reminder","The process will take several seconds or minutes. Please wait patiently")        
            sender = 'test001ceshi@gmail.com'
            host_server = 'smtp.gmail.com'
            pwd = 'dslvhouvwuvtvbvw'
            try:
                mail= self.entry01.get()
                threshold = float(self.entry02.get())
                sec = float(self.entry03.get())*60
            except:
                messagebox.showwarning("Error","Please check the input time and rate!")
            while True:
                i = i+1
                global stop
                if stop:
                    break
                else:
                    print(1)
                    rate = self.getrate()
                    mail_content = self.ctt(rate)
                    receiver = [mail ]#收件人邮箱
                    mail_title = 'Exchange Rate Auto Reminder' #邮件标题
                    msg = MIMEMultipart()# 初始化一个邮件主体
                    msg["Subject"] = Header(mail_title,'utf-8')
                    msg["From"] = sender
                    msg["To"] = Header(str(receiver),'utf-8')# msg['To'] = ";".join(receiver)
                    msg.attach(MIMEText(mail_content,'html','utf-8'))
                    smtp = SMTP_SSL(host_server,465) # ssl登录
                    smtp.login(sender,pwd)
                    # msg：发送消息：邮件内容。一般是msg.as_string():as_string()是将msg(MIMEText对象或者MIMEMultipart对象)变为str。
                    if rate < threshold:
                        try:
                            smtp.sendmail(sender,receiver,msg.as_string())
                            if i == 1:
                                messagebox.showinfo("Reminder","Successfully launched, please minimize the window")
                                smtp.quit()
                                time.sleep(sec)
                            else:
                                smtp.quit()
                                time.sleep(sec)
                        except SMTPRecipientsRefused:
                            messagebox.showwarning("Error","Incorrect email address")
                            smtp.quit()
                            break
                    else:
                        time.sleep(sec)
            

In [3]:
if __name__ == '__main__':
    stopthreads = False
    root = Tk()
    root.title("澳元汇率提醒器")
    root.geometry("600x450+1000+500")
    root.config(bg="#000000")
    app = application(master = root)
    root.mainloop()

1
1
